In [88]:
import os, nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
import pandas as pd
os.getcwd()

'c:\\Users\\T-Gamer\\Documents\\SideDrive\\UFMA\\2022.1\\Topicos Especiais (NLP)\\Exercicios\\Trabalho Final\\Implementação\\source'

In [119]:
def remove_ats_hashtags_and_links(text:str) -> str :
    gross_tokenlist = text.split()
    text = ""
    for gross_token in gross_tokenlist :
        if not (gross_token.startswith("@") or
                gross_token.startswith("#") or
                gross_token.startswith("http")) :
            text += gross_token + " "
    return text[:-1]
def remove_stopwords(tokenlist:list[str]) -> list[str] :
    stoplist = stopwords.words('english')
    newtokenlist = []
    for token in tokenlist :
        if not token in stoplist :
            newtokenlist.append(token)
    return newtokenlist
def remove_nonalpha(tokenlist:list[str]) -> list[str] :
    newtokenlist = []
    for tkn in tokenlist :
        newtkn = ""
        for char in tkn :
            if char.isalpha() :
                newtkn += char
        if newtkn != "" : newtokenlist.append(newtkn)
    return newtokenlist
def preprocess(text:str) -> str :
    text = remove_ats_hashtags_and_links(text)
    text = text.lower()
    text = text.replace("-", " ")
    tokenlist = nltk.word_tokenize(text)
    tokenlist = remove_stopwords(tokenlist)
    lemmatizer = nltk.WordNetLemmatizer()
    lemmalist = [lemmatizer.lemmatize(token) for token in tokenlist]
    lemmalist = remove_nonalpha(lemmalist)
    text = ""
    for tkn in lemmalist :
        text += tkn + " "
    text = text[ : -1]
    if text == "" : return " " # Treat NaN
    return text

In [90]:
# Twiter Airlines
dataset_folder = "../resources/datasets/TwitterAirlines"

dataset = pd.read_csv(f"{dataset_folder}/Tweets.csv", index_col=0)

columns_to_keep = ["airline_sentiment", "airline_sentiment_confidence", "text"]
dataset_clean = dataset.iloc[ : , [column in columns_to_keep for column in dataset.columns]]
dataset_clean.to_csv(f"{dataset_folder}/TweetsDroppedCols.csv")

dataset_clean = dataset_clean.copy() # Defrag

processed_text = []
classes = []
index = []
for i, (_, row) in enumerate(dataset_clean.iterrows()) :
    if i % 100 == 0 : 
        print(i)
    processed = preprocess(row["text"])
    if processed == "" : processed = " "
    processed_text.append(processed)
    classes.append(["negative", "neutral", "positive"].index(row["airline_sentiment"]))
    index.append(row.name)
processed_text_column = pd.Series(processed_text, index, name="text")
classes_column = pd.Series(classes, index, name="class")

dataset_processed = pd.concat([processed_text_column, classes_column], axis=1)
dataset_processed.to_csv(f"{dataset_folder}/TweetsProcessed.csv")
dataset_processed

In [122]:
# Stanford Sentiment Treebank 2 e IMDB Movie Reviews
# Args
dataset_folder = "../resources/datasets/IMDBMovieReviews"
dataset_name = "IMDB_MR"
# dataset_folder = "../resources/datasets/StanfordSentimentTreebank"
# dataset_name = "SST2"
# Exec
dataset = pd.read_csv(f"{dataset_folder}/{dataset_name}.csv", index_col=0)

processed_text = []
index = []
for i, (_, row) in enumerate(dataset.iterrows()) :
    if i % 100 == 0 : 
        print(i)
    processed = preprocess(row["text"])
    if processed == "" : processed = " "
    processed_text.append(processed)
    index.append(row.name)
processed_text_column = pd.Series(processed_text, index, name="text")

dataset_processed = dataset.copy()
dataset_processed["text"] = processed_text_column
dataset_processed.to_csv(f"{dataset_folder}/{dataset_name}Processed.csv")
dataset_processed

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

,label,text
5814_8,1,stuff going moment mj ve started listening mus...
2381_9,1,classic war worlds timothy hines entertaining ...
7759_3,0,film start manager nicholas bell giving welcom...
3630_4,0,must assumed praised film greatest filmed oper...
9495_8,1,superbly trashy wondrously unpretentious s exp...
...,...,...
3453_3,0,seems like consideration gone imdb review film...
5064_1,0,nt believe made film completely unnecessary fi...
10905_3,0,guy loser ca nt get girl need build picked str...
10194_3,0,minute documentary buñuel made early s one spa...
